# Superbowl 2017


## tl;dr

Vamos a analizar una colección de tweets en inglés publicados durante un partido de fútbol.


## Contexto

El pasado 5 de febrero se celebró la [51ª edición de la Superbowl](https://en.wikipedia.org/wiki/Super_Bowl_LI), la gran final del campeonato de fútbol americano de la NFL. El partido enfrentó a los [New England Patriots](https://en.wikipedia.org/wiki/New_England_Patriots) (los favoritos, los de la costa este, con [Tom Brady](https://en.wikipedia.org/wiki/Tom_Brady) a la cabeza) contra los [Atlanta Falcons](https://en.wikipedia.org/wiki/Atlanta_Falcons) (los aspirantes, los del Sur, encabezados por [Matt Ryan](https://en.wikipedia.org/wiki/Matt_Ryan_(American_football))).

![](http://bandageek.com/wp-content/uploads/2017/02/patriots-vs-falcons.jpg)

Como cualquier final, el resultado *a priori* era impredecible y a un partido podía ganar cualquiera. Pero el del otro día fue un encuentro inolvidable porque comenzó con el equipo débil barriendo al favorito y con un Brady que no daba una. Al descanso, el marcador reflejaba un inesperado **3 - 28** y todo indicaba que los Falcons ganarían su primer anillo.

![](https://pbs.twimg.com/media/C38X-Z-VUAA-UAV.jpg)

Pero, en la segunda mitad, Brady resurgió... y su equipo comenzó a anotar una y otra vez... con los Falcons ko. Los Patriots consiguieron darle la vuelta al marcador y vencieron por **34 - 28** su quinta Superbowl. Brady fue elegido MVP del encuentro y aclamado como el mejor quaterback de la historia.

![](http://images.complex.com/complex/images/c_limit,w_680/f_auto,fl_lossy,pg_1,q_auto/d36dh2j3micwoszunssh/tom-brady-new-england-patriots-vince-lombardi-trophy-super-bowl-li)

Como os imaginaréis, tanto vaivén nos va a dar mucho juego a la hora de analizar un corpus de mensajes de Twitter. Durante la primera mitad, es previsible que encuentres mensajes a favor de Atlanta y burlas a New England y a sus jugadores, que no estaban muy finos. Pero al final del partido, con la remontada, las opiniones y las burlas cambiarán de sentido.

Como tanto Tom Brady como su entrenador, Bill Belichick, habían declarado públicamente sus preferencias por Donald Trump durante las elecciones a la presidencia, es muy probable que encuentres mensajes al respecto y menciones a demócratas y republicanos.

Por último, durante el *half time show* actuó Lady Gaga, que también levanta pasiones a su manera, así que es probable que haya menciones a otras *reinas* de la música y comparaciones con actuaciones pasadas.

![](http://www.billboard.com/files/styles/article_main_image/public/media/12-lady-gaga-super-bowl-feb-2017-billboard-1548.jpg)


## Los datos

El fichero `2017-superbowl-tweets.tsv` ubicado en el directorio `data/` contiene una muestra, ordenada cronológicamente, de mensajes escritos en inglés publicados antes, durante y después del partido. Todos los mensajes contienen el hashtag `#superbowl`. Hazte una copia de este fichero en el directorio `notebooks` de tu espacio personal.

El fichero es en realidad una tabla con cuatro columnas separadas por tabuladores, que contiene líneas (una por tweet) con el siguiente formato:

    id_del_tweet fecha_y_hora_de_publicación autor_del_tweet texto_del_mensaje


La siguiente celda te permite abrir el fichero para lectura y cargar los mensajes en la lista `tweets`. Modifica el código para que la ruta apunte a la copia local de tu fichero.

In [ ]:
# !gunzip ../data/2017-twitter-messages.tsv.gz
!ls -l ../data

In [1]:
tweets = []
RUTA = "../data/2017-twitter-messages.tsv"
for line in open(RUTA).readlines():
    tweets.append(line.split("\t"))

In [ ]:
# en Windows, asegúrate de que abres el fichero de texto forzando la codificación para que sea Unicode
import codecs

tweets = []
RUTA = "../data/2017-twitter-messages.tsv"

with codecs.open(RUTA, "r", "utf-8") as f:
    for line in f.readlines():
        tweets.append(line.split("\t"))

Fíjate en la estructura de la lista: se trata de una lista de tuplas con cuatro elementos. Puedes comprobar si el fichero se ha cargado como debe en la siguiente celda:

In [2]:
ultimo_tweet = tweets[-1]
print("id =>", ultimo_tweet[0])
print("fecha =>", ultimo_tweet[1])
print("autor =>", ultimo_tweet[2])
print("texto =>", ultimo_tweet[3])

id => 828498211253997568
fecha => 2017-02-06 06:59:00
autor => ceebrie
texto => Honestly WHAT a SuperBowl performance ✨



## Al lío

A partir de aquí puedes hacer distintos tipos de análisis. Añade tantas celdas como necesites para intentar, por ejemplo:

- calcular distintas estadísticas de la colección: número de mensajes, longitud de los mensajes, presencia de hashtags y emojis, etc.
- número de menciones a usuarios, frecuencia de aparición de menciones, frecuencia de autores
- calcular estadísticas sobre usuarios: menciones, mensajes por usuario, etc.
- calcular estadísticas sobre las hashtags
- calcular estadísticas sobre las URLs presentes en los mensajes
- calcular estadísticas sobre los emojis y emoticonos de los mensajes
- extraer automáticamente las entidades nombradas que aparecen en los mensajes y su frecuencia
- procesar los mensajes para extraer y analizar opiniones: calcular la subjetividad y la polaridad de los mensajes
- extraer las entidades nombradas que levantan más pasiones, quiénes son los más queridos y los más odiados, atendiendo a la polaridad de los mensajes
- comprobar si la polaridad de alguna entidad varía radicalmente a medida que avanza el partido
- cualquier otra cosa que se te ocurra :-P



In [3]:
# escribe tu código aquí

import spacy
nlp = spacy.load("en_core_web_md")

from flair.data import Sentence
from flair.models import SequenceTagger
from flair.models import TextClassifier

pos = SequenceTagger.load("pos-fast")
ner = SequenceTagger.load("ner-fast")
sentiment = TextClassifier.load("en-sentiment")

2021-01-31 16:00:01,778 loading file /home/victor/.flair/models/en-pos-ontonotes-fast-v0.5.pt
2021-01-31 16:00:04,994 loading file /home/victor/.flair/models/en-ner-fast-conll03-v0.4.pt
2021-01-31 16:00:06,261 loading file /home/victor/.flair/models/sentiment-en-mix-distillbert_3.1.pt


In [4]:
# comparamos el reconocedor de entidades de spaCy y flair

for sample in tweets[1000:1200]:
    if len(sample) == 4:
        text = sample[3]
        
        # analizamos el sentimiento con flair
        s = Sentence(text)
        sentiment.predict(s)
        print(f"{text} -> {s.labels}")
        
        # analizamos entidades nombradas con spaCy
        doc = nlp(text)
        spacy_entities = []
        for entity in doc.ents:
            spacy_entities.append((entity, entity.label_))
        print(f"spaCy: {spacy_entities}")

        # analizamos entidades nombradas con flair
        ner.predict(s)
        print(f"""flair: {[entity for entity in s.get_spans("ner")]}\n""")

RT @RyanEFox2: Here is a rundown of the pregame show leading up to kickoff. #SuperBowl @FOX2News https://t.co/qUFdvaece6
 -> [NEGATIVE (0.9902)]
spaCy: [(RT, 'ORG'), (SuperBowl, 'MONEY')]
flair: []

RT @MesquiteBeachVB: Ready to work up a Superbowl appetite at Mesquite Beach! https://t.co/4Fbbxalstq
 -> [POSITIVE (0.9987)]
spaCy: [(Superbowl, 'DATE'), (Mesquite Beach, 'LOC')]
flair: [<LOC-span (13,14): "Mesquite Beach">]

RT @AltUSFWS: Trumps new Executive Order will issue decree that he alone won the super bowl. He had the largest crowd in football history #…
 -> [NEGATIVE (0.5123)]
spaCy: [(RT, 'ORG'), (the super bowl, 'EVENT')]
flair: [<ORG-span (7,8): "Executive Order">]

a 00h ya le superbowl
 -> [POSITIVE (0.9995)]
spaCy: [(00h, 'ORDINAL')]
flair: []

RT @SimpsonsQOTD: "Yeah, ever since I was a boy I've always loved the...*Atlanta Falcons*." #SuperBowl https://t.co/TVkLiRzcHs
 -> [POSITIVE (0.914)]
spaCy: [(RT, 'ORG'), (SuperBowl, 'DATE')]
flair: [<ORG-span (20,21): "Atlanta Falc

In [5]:
# imprimimos el sentimiento asociado a los mensajes que contienen entidades de tipo persona
for tweet in tweets[1200:1400]:
    if len(tweet) == 4:
        # comprobamos que la línea está bien formada, había problemillas con algunas filas
        s = Sentence(tweet[3])

        # pasamos el reconocedor de entidades
        ner.predict(s)

        # pasamos el clasificador de sentimiento
        sentiment.predict(s)

        # iteramos por las entidades reconocidas
        for entity in s.get_spans("ner"):
            if entity.tag == "PER":
                # si la entidad es persona, imprimimos el mensaje, la entidad y el sentimiento
                print(tweet[3][:-1])
                print(entity, s.labels, "\n")

RT @thebrewergroup: #throwback to our #2014 #superbowl #watchparty with Ralph Kader and Gerald Commissiong ! Join us tonight, 9th Annual Wa…
Span [16,17]: "Ralph Kader"   [− Labels: PER (0.9519)] [POSITIVE (0.9993)] 

RT @thebrewergroup: #throwback to our #2014 #superbowl #watchparty with Ralph Kader and Gerald Commissiong ! Join us tonight, 9th Annual Wa…
Span [19,20]: "Gerald Commissiong"   [− Labels: PER (0.9023)] [POSITIVE (0.9993)] 

RT @espn: Tom Brady's mom has been battling illness. His dad has only been to one game this year.
Span [5,6]: "Tom Brady"   [− Labels: PER (0.9864)] [NEGATIVE (0.6147)] 

@RanaHarbi @Transiently God would rather watch Superbowl.
Span [5]: "God"   [− Labels: PER (0.8911)] [POSITIVE (0.6259)] 

RT @holyfag: Let's appreciate Kim's #Superbowl commercial where she gave us the iconic "tragic" line https://t.co/e7b0VRc43H
Span [8]: "Kim"   [− Labels: PER (0.9707)] [NEGATIVE (0.6476)] 

"The best players in the country come from the state of Texas." - Jimmy J